<a href="https://colab.research.google.com/github/athens-21/DE242/blob/main/Google_Stock_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Programmer** : Rattanin Kliangmol


**conclusion**

After evaluating the performance of different forecasting models (Simple Moving Average, Simple Exponential Smoothing, and Holt-Winters Exponential Smoothing) using the MAE, MSE, RMSE, and MAPE metrics, I conclusion for

**the best performance is**
- **Holt-Winters Exponential Smoothin** performed the best across all metrics because the number of the MAE, MSE, RMSE, and MAPE lower than any forecasting models:
  - MAE: 14.40
  - MSE: 491.13
  - RMSE: 22.16
  - MAPE: 1.14%


In [17]:
import kagglehub

path = kagglehub.dataset_download("shreenidhihipparagi/google-stock-prediction")
print("Path to dataset files:", path)


Path to dataset files: /root/.cache/kagglehub/datasets/shreenidhihipparagi/google-stock-prediction/versions/2


In [2]:
import pandas as pd

df = pd.read_csv('/root/.cache/kagglehub/datasets/shreenidhihipparagi/google-stock-prediction/versions/2/GOOG.csv')
df.head()


,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,GOOG,2016-06-14 00:00:00+00:00,718.27,722.47,713.1200,716.48,1306065,718.27,722.47,713.1200,716.48,1306065,0.0,1.0
1,GOOG,2016-06-15 00:00:00+00:00,718.92,722.98,717.3100,719.00,1214517,718.92,722.98,717.3100,719.00,1214517,0.0,1.0
2,GOOG,2016-06-16 00:00:00+00:00,710.36,716.65,703.2600,714.91,1982471,710.36,716.65,703.2600,714.91,1982471,0.0,1.0
3,GOOG,2016-06-17 00:00:00+00:00,691.72,708.82,688.4515,708.65,3402357,691.72,708.82,688.4515,708.65,3402357,0.0,1.0
4,GOOG,2016-06-20 00:00:00+00:00,693.71,702.48,693.4100,698.77,2082538,693.71,702.48,693.4100,698.77,2082538,0.0,1.0


In [5]:
selected_df = df[['date', 'close']]
selected_df['date'] = pd.to_datetime(selected_df['date'])  # Convert to datetime
selected_df.set_index('date', inplace=True)

<ipython-input-5-d92fb2fff228>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['date'] = pd.to_datetime(selected_df['date'])  # Convert to datetime


In [6]:
selected_df.head()

,close
date,
2016-06-14 00:00:00+00:00,718.27
2016-06-15 00:00:00+00:00,718.92
2016-06-16 00:00:00+00:00,710.36
2016-06-17 00:00:00+00:00,691.72
2016-06-20 00:00:00+00:00,693.71


# **Apply forecasting techniques to the dataset**

**Simple Moving Average (SMA)**

In [8]:
selected_df['SMA'] = selected_df['close'].rolling(window=30).mean()

<ipython-input-8-71011c9bd076>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['SMA'] = selected_df['close'].rolling(window=30).mean()


**Simple Exponential Smoothing (SES)**

In [10]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

model_ses = SimpleExpSmoothing(selected_df['close'])
model_ses_fit = model_ses.fit(smoothing_level=0.2, optimized=False)
selected_df['SES'] = model_ses_fit.fittedvalues

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
<ipython-input-10-d86ae027800a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['SES'] = model_ses_fit.fittedvalues


**Holt-Winters Exponential Smoothingt**

In [12]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

model_hw = ExponentialSmoothing(selected_df['close'], trend='add', seasonal='add', seasonal_periods=12)
model_hw_fit = model_hw.fit()
selected_df['Holt-Winters'] = model_hw_fit.fittedvalues

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
<ipython-input-12-16bc3b8c48b3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['Holt-Winters'] = model_hw_fit.fittedvalues


# **Evaluate the performance**

In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np

In [15]:
mae_sma = mean_absolute_error(selected_df['close'][30:], selected_df['SMA'][30:])
mse_sma = mean_squared_error(selected_df['close'][30:], selected_df['SMA'][30:])
rmse_sma = np.sqrt(mse_sma)
mape_sma = mean_absolute_percentage_error(selected_df['close'][30:], selected_df['SMA'][30:])

mae_ses = mean_absolute_error(selected_df['close'][30:], selected_df['SES'][30:])
mse_ses = mean_squared_error(selected_df['close'][30:], selected_df['SES'][30:])
rmse_ses = np.sqrt(mse_ses)
mape_ses = mean_absolute_percentage_error(selected_df['close'][30:], selected_df['SES'][30:])

mae_hw = mean_absolute_error(selected_df['close'][30:], selected_df['Holt-Winters'][30:])
mse_hw = mean_squared_error(selected_df['close'][30:], selected_df['Holt-Winters'][30:])
rmse_hw = np.sqrt(mse_hw)
mape_hw = mean_absolute_percentage_error(selected_df['close'][30:], selected_df['Holt-Winters'][30:])


# Solution of modal

In [16]:
print(f"Simple Moving Average (SMA) - MAE: {mae_sma}, MSE: {mse_sma}, RMSE: {rmse_sma}, MAPE: {mape_sma}")
print(f"Simple Exponential Smoothing (SES) - MAE: {mae_ses}, MSE: {mse_ses}, RMSE: {rmse_ses}, MAPE: {mape_ses}")
print(f"Holt-Winters Exponential Smoothing - MAE: {mae_hw}, MSE: {mse_hw}, RMSE: {rmse_hw}, MAPE: {mape_hw}")

Simple Moving Average (SMA) - MAE: 46.29358333333334, MSE: 4267.05775875439, RMSE: 65.32272008079876, MAPE: 0.03570754457296335
Simple Exponential Smoothing (SES) - MAE: 24.468344245339196, MSE: 1224.3260027916283, RMSE: 34.990370143678504, MAPE: 0.019155395183064073
Holt-Winters Exponential Smoothing - MAE: 14.398011085247061, MSE: 491.1266569611178, RMSE: 22.161377596194644, MAPE: 0.011420617883396293
